In [ ]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2024-03-13 19:08:24--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0.001s  

2024-03-13 19:08:24 (18.3 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]

--2024-03-13 19:08:24--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP re

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [ ]:
input_files = [
    'edgar_allan_poe.txt',
    'robert_frost.txt'
]

In [ ]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [ ]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


We are treating each line as an input sample. This is opposed to an entire verse or an entire poem, so we will begin by creating two empty lists, one to hold the text and the other the label.

In [ ]:
# collect data into lists
input_texts = []
labels = []

# loop through each input file
for label, f in enumerate(input_files):
  print(f"{f} correspondes to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

robert_frost.txt correspondes to label 0
edgar_allan_poe.txt correspondes to label 1


In [ ]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [ ]:
len(Ytrain), len(Ytest)

(1615, 539)

In [ ]:
train_text[:5]

['if i remember rightly it had sprung',
 'sweet flowers ere long',
 'for the resurrection of deepburied faith',
 'thy grief thy joy thy hate thy love',
 'alas i cannot feel for tis not feeling']

In [ ]:
Ytrain[:5]

[0, 1, 1, 1, 1]

This step is to convert our text into integers. These will be indexes into the Markov Matrices we are about to define.

We begin setting idx=1 which will act as out current index as we loop through the  text.

We'll also initialize a word to index dictionary with one entry, which maps the unknown token to zero. This will never be used for the train set, but might be used for the test set if the test set contain words that do not appear in the train set.

In [ ]:
idx = 1
word2idx = {'<unk>': 0}

In [ ]:
# populate word2idx
for text in train_text:
  # split the text into tokens
  tokens = text.split()
  for token in tokens:
    if token not in word2idx:
      word2idx[token] = idx
      idx += 1

In [ ]:
word2idx

{'<unk>': 0,
 'if': 1,
 'i': 2,
 'remember': 3,
 'rightly': 4,
 'it': 5,
 'had': 6,
 'sprung': 7,
 'sweet': 8,
 'flowers': 9,
 'ere': 10,
 'long': 11,
 'for': 12,
 'the': 13,
 'resurrection': 14,
 'of': 15,
 'deepburied': 16,
 'faith': 17,
 'thy': 18,
 'grief': 19,
 'joy': 20,
 'hate': 21,
 'love': 22,
 'alas': 23,
 'cannot': 24,
 'feel': 25,
 'tis': 26,
 'not': 27,
 'feeling': 28,
 'rivers': 29,
 'glide': 30,
 'and': 31,
 'then': 32,
 'you': 33,
 'will': 34,
 'find': 35,
 'your': 36,
 'money': 37,
 'in': 38,
 'creases': 39,
 'now': 40,
 'as': 41,
 'night': 42,
 'was': 43,
 'senescent': 44,
 'with': 45,
 'whose': 46,
 'vast': 47,
 'wheels': 48,
 'being': 49,
 'cut': 50,
 'off': 51,
 'from': 52,
 'friends': 53,
 'help': 54,
 'out': 55,
 'this': 56,
 'beadwork': 57,
 'what': 58,
 'can': 59,
 'no': 60,
 'holy': 61,
 'rays': 62,
 'heaven': 63,
 'come': 64,
 'down': 65,
 'warren': 66,
 'at': 67,
 'march': 68,
 'meeting': 69,
 'reason': 70,
 'to': 71,
 'carry': 72,
 'again': 73,
 'see': 74,


In [ ]:
len(word2idx)

2555

Convert the data into integer format. This is what we need to index our A and oour Pi.

In [ ]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  """
  The difference here is that it is possible that not every word appear in the
  test set, so we can't only try to index to our idx dictionary. Instead we use
  get function and when we can't find the word, we return 0. This corresponds to
  the unknown token
  """
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [ ]:
train_text_int[100:105]

[[149, 365, 366],
 [182, 43, 60, 367, 368, 369],
 [260, 370, 371, 52, 372, 373],
 [374, 375, 376, 374, 377, 378],
 [115, 13, 379, 182, 31, 380, 31, 13, 381]]

The zeroes in the test set are unknown words.

In [ ]:
test_text_int[100:105]

[[32, 181, 0, 71, 181],
 [666, 58, 667, 335, 500, 171, 100],
 [100, 148, 36, 850, 31, 36, 0, 280],
 [214, 128, 786, 778, 693, 0, 115, 41, 71, 2197],
 [1384, 31, 0, 441, 724, 1014, 71, 490, 38]]

Now we are ready to build our A and Pi matrices which will represent our Markov Model.

It's important to remember that we don't just have 1 Markov Model, we'll have as many as there are classes. Since we have 2 classes, we'll have 2 As and 2 Pis.

In [ ]:
# initialize A and Pi matrices - for both classes

# lenght of idx: the vocabulary size
V = len(word2idx)
"""
We initiliaze each of these arrays to all ones. The reason for that is that we
are going to be using Add-One Smoothing, so these 1s are the initial fake counts
for each initial word and each transition.
"""
A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

Now we populate the As and the Pis with the appropriate counts from the training set.

In [ ]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  # here each element is a line of a poem represent by a list of ints
  for tokens in text_as_int:
    # This variable will help us keep track if we are populating A or Pi
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sequence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition (one word to the next)
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx

# only the samples of class 0
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
# only the samples of class 1
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

Now we normalize A and Pi so they are valid probability matrices.


In [ ]:
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /=  pi0.sum()
"""
keepdims = True ensures that the sum is still 2D, which is required for the
division to broadcast correctly  in numpy.

For Pi there is no need to do this since it's just a 1D array.
"""
A1 /= A1.sum(axis=1, keepdims=True)
pi1 /=  pi1.sum()

In [ ]:
# Log A and Pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [ ]:
# compute priors
# How many samples belong to class zero and one in the training set.
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
# compute the prior probabilites, p0 and p1
p0 = count0  / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.6674922600619195, 0.33250773993808047)

In [ ]:
# Build a classifier
class Classifier:
  # constructor
  def __init__(self, logAs, logPis, logPriors):
    self.logAs = logAs
    self.logPis = logPis
    self.logPriors = logPriors
    self.K = len(logPriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logPi = self.logPis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logPi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input, c) + self.logPriors[c] \
                    for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [ ]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [ ]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain==Ytrain)}")

Train acc: 0.9944272445820433


In [ ]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8051948051948052


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[1078,    0],
       [   9,  528]])

In [ ]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[346,  12],
       [ 93,  88]])

In [ ]:
f1_score(Ytrain, Ptrain)

0.9915492957746479

In [ ]:
f1_score(Ytest, Ptest)

0.6263345195729537